## Étude suggestion métier vers code rome
l'API matching utilisait jusqu'à présent l'API LBB pour la suggestion métier / code rome.

Vu le besoin en performances accrus, le moment est venu d'intégrer cette fonctionnalite dans l'API matching. Ce notebook sert de base de travail à cet effet.

### Améliorations:
- accent folding & nettoyage pour recherche normalisée
- drop_duplicates pour éviter les retours mulitples identiques
- Ajout score ordre / tri

### Notes:
Nettoyage INPUT: https://machinelearningmastery.com/clean-text-machine-learning-python/

In [35]:
## Chargement des labels rome, ogr
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# Rome
rome_labels = pd.read_csv('../ressources/liste_rome_LBB.csv', sep=',', encoding="utf-8")
rome_labels.columns = ['rome', 'rome_1', 'rome_2', 'rome_3', 'label', 'slug']
print(f"Obtained {len(rome_labels)} ROME labels")
display(HTML(rome_labels.head(5).to_html()))

# OGR
ogr_labels = pd.read_csv('../ressources/list_OGR_LBB.csv', sep=',', encoding="utf-8")
ogr_labels.columns = ['code', 'rome_1', 'rome_2', 'rome_3', 'label', 'rome']
print(f"Obtained {len(ogr_labels)} OGR labels")
display(HTML(ogr_labels.head(5).to_html()))


Obtained 531 ROME labels


,rome,rome_1,rome_2,rome_3,label,slug
0,A1101,A,11,1,Conduite d'engins agricoles et forestiers,conduite-d-engins-agricoles-et-forestiers
1,A1201,A,12,1,Bûcheronnage et élagage,bucheronnage-et-elagage
2,A1202,A,12,2,Entretien des espaces naturels,entretien-des-espaces-naturels
3,A1203,A,12,3,Entretien des espaces verts,entretien-des-espaces-verts
4,A1204,A,12,4,Protection du patrimoine naturel,protection-du-patrimoine-naturel


Obtained 10948 OGR labels


,code,rome_1,rome_2,rome_3,label,rome
0,11987,A,11,1,Chauffeur / Chauffeuse de machines agricoles,A1101
1,12862,A,11,1,Conducteur / Conductrice d'abatteuses,A1101
2,38874,A,11,1,Conducteur / Conductrice d'automoteur de récolte,A1101
3,13232,A,11,1,Conducteur / Conductrice de machines à vendanger,A1101
4,38878,A,11,1,Conducteur / Conductrice de matériels de semis,A1101


In [37]:
import string

def words_get(raw_query):
    # Split
    words = raw_query.split()
    # Remove punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]
    # Lowercase
    words = [word.lower() for word in words]
    
    return words

def result_build(rome, rome_label, rome_slug, ogr_label=None):
    if ogr_label:
        label = f"{rome_label} ({ogr_label}, ...)"
    else:
        label = rome_label
    return {
        'id': rome,
        'label': label,
        'value': value,
        'occupation': rome_slug,
        'score': 5
    }

def rome_suggest(query, rome_df, ogr_df):
    words = words_get(query)
    rome_raw_matches = []
    ogr_raw_matches = []
    for word in words:
        rome_raw_matches.append( rome_df[rome_de['label'].str.contains(word)] )
        ogr_raw_matches.append( ogr_df[ogr_df['label'].str.contains(word)] )
    rome_matches = pd.concat(rome_raw_matches)
    ogr_matches = pd.concat(ogr_raw_matches)
    
    results = {}
    for _i, rome in rome_matches.iterrows():
        results[rome['rome']] = result_build(
            rome['rome'],
            rome['label'],
            rome['slug']
        )
        
    for _i, ogr in ogr_matches.iterrows():
        
        
    return rome_matches

In [34]:

rome_suggest("entretien direction", rome_labels, ogr_labels)

,rome,rome_1,rome_2,rome_3,label,slug
298,I1101,I,11,1,Direction et ingénierie en entretien infrastru...,direction-et-ingenierie-en-entretien-infrastru...
300,I1103,I,11,3,Supervision d'entretien et gestion de véhicules,supervision-d-entretien-et-gestion-de-vehicules
322,I1604,I,16,4,Mécanique automobile et entretien de véhicules,mecanique-automobile-et-entretien-de-vehicules
185,G1401,G,14,1,Assistance de direction d'hôtel-restaurant,assistance-de-direction-d-hotel-restaurant
471,M1604,M,16,4,Assistanat de direction,assistanat-de-direction


In [49]:
query="cariste"
q=f'.*{query}.*'

res = ogr_labels[ogr_labels['label'].str.contains(query)]
results = []
for index, row in res.iterrows():
    rome = row['rome']
    label = row['label']
    romes = rome_labels[rome_labels['rome'] == rome]
    for index, rome_row in romes.iterrows():
        results.append({
            'code': rome,
            'label': f"{rome_row['label']} ({label}, ...)"
        })
results

[{'code': 'N1101',
  'label': "Conduite d'engins de déplacement des charges (Agent / Agente cariste de livraison ferroviaire, ...)"},
 {'code': 'N1101',
  'label': "Conduite d'engins de déplacement des charges (Chef cariste, ...)"},
 {'code': 'N1103',
  'label': 'Magasinage et préparation de commandes (Magasinier / Magasinière cariste, ...)'}]